In [1]:
# Author Emmanuel Sedicol
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imutils

DEBUG = False

# Uninverted/ Inverted 2D Court (x,y) Coordinates

Four Corners:
- bottom left, top left, bottom right, top right
- uninvert - [(0, 400), (0, 0), (600, 400), (600, 0)]
- invert - [(0, 400), (0, 0), (600, 400), (600, 0)]

Free Throw Section:
- bottom left, top left, bottom right, top right
- uninvert - [(226, 400), (226, 240), (394, 400), (394, 240)]
- invert - [(226, 160), (226, 0), (370, 160), (370, 0)]

Three Point Arc Section 
- left, center, right
- uninvert - [(63, 400), (300, 185), (535, 400)]
- invert - [(63, 0), (300, 216), (535, 0)]

# Player Position Extraction using Using Hough Transform Algorithm 

In [2]:
if DEBUG:
    plt.figure(figsize=(10,8))

    pts_3D = np.array([(0,235), (70, 236), (230, 235), (370, 232), (530, 230), (600, 227), (300, 360), (200, 315), (405, 310)])
    pts_2D = np.array([(0, 0), (63, 0),(226, 0), (370, 0), (535, 0), (600, 0), (300, 216), (226, 160), (370, 160) ])

    frame = cv2.imread('images/court/court_extraction.png')
    frame  = cv2.resize(frame, (600,400))

    court = cv2.imread('images/court/court_invert.png')
    court  = cv2.resize(court, (600,400))

    roi = frame[150:400, 0: 600]

    r_h, r_w, r_c = roi.shape
    i_h, i_w, i_c = frame.shape

    # line detection works better with grayscale images (less pixel range to process)
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)

    # Canny edge detection to detect line edges
    edges = cv2.Canny(roi, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

    # calculate (x1,y1) and (x2, y2) coordinates
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho

        # extend lines out 
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))

        cv2.line(frame, (x1, y1 + (i_h - r_h)), (x2, y2 + (i_h - r_h)), (255, 0 ,100), 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
    fig.suptitle("PLAYER POSITION EXTRACTION")

    ax1.set_title("3D Image")
    for p in range(0, len(pts_3D)):
        ax1.scatter(pts_3D[p][0], pts_3D[p][1], s=100, c='r', marker='o')

    ax2.set_title("2D Image")
    for p in range(0, len(pts_2D)):
        ax2.scatter(pts_2D[p][0], pts_2D[p][1], s=200, c='r')

    ax1.imshow(frame)
    ax2.imshow(court)
    plt.show()

# Function for locating and mapping the positon of a person

In [3]:
def estimate_position(frame, court, pts_3D, pts_2D):
    x = 0
    y = 0
    # yellow lower and upper range
    lower_range = (10, 160, 160)                       
    upper_range = (100,255,255)  
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # morph points from 3D frame onto the corrensponding point located on the 2D court image
    matrix, status = cv2.findHomography(pts_3D, pts_2D)
    warped_frame = cv2.warpPerspective(frame, matrix, (court.shape[1], court.shape[0]))
    
    # image filtering for a better color detection
    image = cv2.cvtColor(warped_frame, cv2.COLOR_BGR2RGB)
    blur = cv2.GaussianBlur(image, (15, 15),0)
    erode = cv2.erode(blur, None, iterations=2)
    dilate = cv2.dilate(erode, None, iterations=2)
    hsv = cv2.cvtColor(dilate, cv2.COLOR_BGR2HSV)
    
    # using the yellow color range we create a mask retrieving objects that contains values between the range
    mask = cv2.inRange(hsv, lower_range, upper_range)  

    # retrieve contours from mask
    cnts = cv2.findContours(mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    if  len(cnts) > 0: 
        # using the max contour value we create a minimum enclosing circle on the contour to get an estimation of the center points
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)

        if radius > 5:
            # mark positions
            court = cv2.circle(court.copy(), (int(x), int(y)), 3, (0, 255, 100), 3) 
            cv2.putText(court, f'(x={round(x , 1)}, y={round(y, 1)})', (int(x - 90), int(y + 35)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (25, 255, 25), 2 )

    return warped_frame, mask, court, x, y

# Test position estimation function

In [4]:
if DEBUG:
    frame2 = cv2.imread('/images/court/court_extraction.png')
    frame2  = cv2.resize(frame, (600,400))
    warped_frame, mask, court, _, _ =  estimate_position(frame2, court, pts_3D, pts_2D)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8))

    ax1.set_title("Warped Image")
    ax2.set_title("Mask")
    ax3.set_title("Estimate Player Position")

    ax1.imshow(warped_frame)
    ax2.imshow(mask)
    ax3.imshow(cv2.cvtColor(court, cv2.COLOR_BGR2RGB))
    plt.show()